In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
data_train = pd.read_csv("data/train.csv")
df_train = data_train.copy()
data_train.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [66]:
data_train.shape

(1460, 81)

In [16]:
nan_cols = df_train.isnull().sum().index[df_train.isnull().sum()>0.15*len(df_train)].to_list()
nan_cols

['LotFrontage',
 'Alley',
 'MasVnrType',
 'FireplaceQu',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [17]:
df_train = df_train.drop(nan_cols, axis=1)

In [20]:
df_train = df_train.drop('Id', axis=1)

In [22]:
df_train.select_dtypes('object').columns

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [23]:
df_train.select_dtypes('int').columns

Index(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [24]:
df_train.select_dtypes('float').columns

Index(['MasVnrArea', 'GarageYrBlt'], dtype='object')

In [26]:
df_train.isnull().sum()[df_train.isnull().sum()>0.0*len(df_train)]

MasVnrArea       8
BsmtQual        37
BsmtCond        37
BsmtExposure    38
BsmtFinType1    37
BsmtFinType2    38
Electrical       1
GarageType      81
GarageYrBlt     81
GarageFinish    81
GarageQual      81
GarageCond      81
dtype: int64

In [34]:
df_train[df_train['GarageType'].isnull()][['GarageType', 'GarageYrBlt', 'GarageFinish','GarageQual', 'GarageCond' ]]

,GarageType,GarageYrBlt,GarageFinish,GarageQual,GarageCond
39,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN
78,NaN,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,NaN,NaN
89,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
1349,NaN,NaN,NaN,NaN,NaN
1407,NaN,NaN,NaN,NaN,NaN
1449,NaN,NaN,NaN,NaN,NaN
1450,NaN,NaN,NaN,NaN,NaN


In [38]:
df_train[[ 'GarageCond' ]].value_counts()

GarageCond
TA            1326
Fa              35
Gd               9
Po               7
Ex               2
Name: count, dtype: int64

In [59]:
def imputer(df):
    for column in df.columns:
        if 'Garage' in column:
            df[column]=df[column].fillna('NO_Garage')
        if 'Bsmt' in column:
            df[column]=df[column].fillna('NO_Basement')
    return df


In [60]:
df_train = imputer(df_train)


In [61]:
df_train.isnull().sum()[df_train.isnull().sum()>0.0*len(df_train)]

MasVnrArea    8
Electrical    1
dtype: int64

In [63]:
# drop rows that have NaN values
df_train = df_train.drop_duplicates()
df_train = df_train.dropna(axis=0)

In [65]:
df_train.shape

(1451, 73)

In [68]:
df_train.isnull().sum().sum()

0

## ML model 

In [83]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor

In [101]:
X = df_train.drop('SalePrice', axis=1)
y = df_train['SalePrice'].to_list()

In [102]:
X_train, X_test, y_train,  y_test = train_test_split(X, y, shuffle = True ,random_state = 42)

In [103]:
y_train.shape

AttributeError: 'list' object has no attribute 'shape'

In [106]:
list_of_models= {#"AdaBoost":AdaBoostRegressor(random_state = 42), 
"RandomForest":RandomForestRegressor(random_state = 42),
"GradientBoost":GradientBoostingRegressor(random_state = 42)}

In [108]:
for name, model in list_of_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = model.score(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(f'{model} : R2 score = {r2} and RMSE = {rmse}')


ValueError: could not convert string to float: 'RL'

In [105]:
y_train

[175000,
 124000,
 130000,
 128000,
 141500,
 375000,
 130500,
 190000,
 113000,
 205000,
 137500,
 239000,
 179500,
 125500,
 239000,
 160000,
 140000,
 423000,
 394432,
 88000,
 193000,
 163000,
 128000,
 200000,
 235000,
 184750,
 144152,
 195400,
 173000,
 128500,
 171000,
 187000,
 153900,
 145000,
 89471,
 115000,
 180000,
 253000,
 135000,
 208900,
 135000,
 555000,
 235000,
 274900,
 203000,
 115000,
 103600,
 170000,
 154000,
 129500,
 222500,
 325000,
 244600,
 158500,
 133000,
 131000,
 179400,
 236500,
 100000,
 446261,
 185900,
 150000,
 120500,
 284000,
 242000,
 130000,
 181000,
 188000,
 174000,
 104900,
 145000,
 139000,
 116000,
 129000,
 168500,
 138000,
 209500,
 254900,
 220000,
 190000,
 176000,
 133000,
 143000,
 160000,
 133700,
 111250,
 185000,
 225000,
 135000,
 271900,
 169900,
 110000,
 155000,
 214000,
 106000,
 250000,
 178740,
 82000,
 145000,
 123000,
 109900,
 109500,
 451950,
 60000,
 119500,
 197900,
 129900,
 91300,
 215000,
 94000,
 174900,
 260000